In [22]:
import os 
import numpy as np
import pandas as pd

##Load Data
sub_path="sub31"
all_files=os.listdir(sub_path)
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "" + str(x)[:-4], all_files))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,21stack_mean,22stack_mean,22stack_mean_pro17,22stack_median,23stack_mean
0,5941774d,0.002070,0.001357,0.001357,0.001000,0.001312
1,4023181e,0.209825,0.150446,0.150446,0.209825,0.157868
2,b20200e4,0.000923,0.000308,0.000000,0.000000,0.000269
3,e7f018bb,0.999560,0.999796,1.000000,0.999914,0.999811
4,4371c8c3,0.100764,0.064310,0.064310,0.064310,0.064310


In [23]:
concat_sub.corr()


,21stack_mean,22stack_mean,22stack_mean_pro17,22stack_median,23stack_mean
21stack_mean,1.000000,0.998399,0.998164,0.997546,0.998385
22stack_mean,0.998399,1.000000,0.999887,0.999150,0.999987
22stack_mean_pro17,0.998164,0.999887,1.000000,0.999113,0.999883
22stack_median,0.997546,0.999150,0.999113,1.000000,0.999350
23stack_mean,0.998385,0.999987,0.999883,0.999350,1.000000


In [24]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:].median(axis=1)

In [25]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.95
cutoff_hi = 0.05

In [26]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv(sub_path[-2:]+'stack_mean.csv', 
                                        index=False, float_format='%.6f')

In [27]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv(sub_path[-2:]+'stack_median.csv', 
                                        index=False, float_format='%.6f')

In [28]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv(sub_path[-2:]+'stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

In [29]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv(sub_path[-2:]+'stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

In [30]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
#concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv(sub_path[-2:]+'stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

In [31]:
sub_base = pd.read_csv('22stack_mean_pro17.csv')
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
#concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv(sub_path[-2:]+'stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')